# [Serialization](https://docs.ray.io/en/latest/serialization.html)
Since Ray processes do not share memory space, data transferred between workers and nodes will need to serialized and deserialized. 

Ray uses the [**Plasma object store**](https://arrow.apache.org/docs/python/plasma.html) to efficiently transfer objects across different processes and different nodes. 

Numpy arrays in the object store are shared between workers on the same node (zero-copy deserialization).

Ray has decided to use a **customized Pickle protocol version 5** backport to replace the original PyArrow serializer. This gets rid of several previous limitations (e.g. cannot serialize recursive objects).

Ray is currently compatible with Pickle protocol version 5, while Ray supports serialization of a wider range of objects **(e.g. lambda & nested functions, dynamic classes) with the help of cloudpickle.**



In [1]:
import logging
import ray

## Usually most of self defined class were serizable and needn't any change

In [2]:
import numpy as np
obj = [np.zeros(42)] * 99
l = ray.get(ray.put(obj))
assert l[0] is l[1]  # no problem!

2021-12-28 16:32:36,332	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
2021-12-28 16:32:36,346	WARNING services.py:1826 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 66994176 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.86gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


In [3]:
import time

# `la` in taiwanese means stir

@ray.remote
class Cup:
    def __init__(self, capacity):
        self.capacity = capacity
    
    def la_with_laable_item(self, laable_item):
        if laable_item.la():
            time.sleep(2)
            self.capacity -= 1
            
    def is_good(self):
        return self.capacity <= 0

In [4]:
class Spoon:
    def __init__(self, sp_id):
        self.sp_id = sp_id
    
    def la(self):
        return True
    

class Leborn:
    def __init__(self, lb_id):
        self.lb_id = lb_id
    
    def la(self):
        return False

### What if we `La` with Spoon?


In [5]:
%%time
cup1 = Cup.remote(3)
sp_ref = ray.put(Spoon("good good spoon"))
print('la first time')
cup1.la_with_laable_item.remote(sp_ref)
print('la first time sent')
print(f"is it good drink, {ray.get(cup1.is_good.remote())}")

print('la second time')
cup1.la_with_laable_item.remote(sp_ref)
print('la second time sent')
print(f"is it good drink, {ray.get(cup1.is_good.remote())}")

print('la third time')
cup1.la_with_laable_item.remote(sp_ref)
print('la third time sent')
print(f"is it good drink, {ray.get(cup1.is_good.remote())}")

la first time
la first time sent
is it good drink, False
la second time
la second time sent
is it good drink, False
la third time
la third time sent
is it good drink, True
CPU times: user 279 ms, sys: 98.1 ms, total: 377 ms
Wall time: 6.04 s


In [6]:
%%time
cup2 = Cup.remote(3)

local_sp = Spoon("good good spoon")
print('la first time')
cup2.la_with_laable_item.remote(local_sp)
print('la first time sent')
print(f"is it good drink, {ray.get(cup2.is_good.remote())}")

print('la second time')
cup2.la_with_laable_item.remote(local_sp)
print('la second time sent')
print(f"is it good drink, {ray.get(cup2.is_good.remote())}")

print('la third time')
cup2.la_with_laable_item.remote(local_sp)
print('la third time sent')
print(f"is it good drink, {ray.get(cup2.is_good.remote())}")

la first time
la first time sent
is it good drink, False
la second time
la second time sent
is it good drink, False
la third time
la third time sent
is it good drink, True
CPU times: user 286 ms, sys: 129 ms, total: 415 ms
Wall time: 6.03 s


after spent 6 seconds, cup good good

### What if we `La` with Leborn?

In [7]:
%%time

local_lb = Leborn("good good Leborn")
cup3 = Cup.remote(3)

print('la first time')
cup3.la_with_laable_item.remote(local_lb)
print('la first time sent')
print(f"is it good drink, {ray.get(cup3.is_good.remote())}")

print('la second time')
cup3.la_with_laable_item.remote(local_lb)
print('la second time sent')
print(f"is it good drink, {ray.get(cup3.is_good.remote())}")

print('la third time')
cup3.la_with_laable_item.remote(local_lb)
print('la third time sent')
print(f"is it good drink, {ray.get(cup3.is_good.remote())}")

la first time
la first time sent
is it good drink, False
la second time
la second time sent
is it good drink, False
la third time
la third time sent
is it good drink, False
CPU times: user 16.6 ms, sys: 4.15 ms, total: 20.8 ms
Wall time: 24.8 ms


cup is not good yet, of-course, Leborn is not Laable

## Some Special Class need thier own serialziation

### first way: __ reduce __

In [8]:
import ray
import threading

class A:
    def __init__(self, x):
        self.x = x
        self.lock = threading.Lock()  # could not be serialized!
    

ray.get(ray.put(A(1)))  # fail!


TypeError: can't pickle _thread.lock objects

In [10]:
class A_with_reduce:
    def __init__(self, x):
        self.x = x
        self.lock = threading.Lock()  # could not be serialized!

    def __reduce__(self):
        deserializer = A_with_reduce
        serialized_data = (self.x,)
        return deserializer, serialized_data

ray.get(ray.put(A_with_reduce(1)))  # success!

### second way: register_serializer

In [ ]:
import ray
import threading

class A:
    def __init__(self, x):
        self.x = x
        self.lock = threading.Lock()  # could not be serialized!

ray.get(ray.put(A(1)))  # fail!

def custom_serializer(a):
    return a.x

def custom_deserializer(b):
    return A(b)

# Register serializer and deserializer for class A:
ray.util.register_serializer(
  A, serializer=custom_serializer, deserializer=custom_deserializer)
ray.get(ray.put(A(1)))  # success!

# You can deregister the serializer at any time.
ray.util.deregister_serializer(A)
ray.get(ray.put(A(1)))  # fail!

# Nothing happens when deregister an unavailable serializer.
ray.util.deregister_serializer(A)

### third way: SerializationHelperForA

In [ ]:
import threading

class A:
    def __init__(self, x):
        self.x = x
        self.lock = threading.Lock()  # could not serialize!

ray.get(ray.put(A(1)))  # fail!

class SerializationHelperForA:
    """A helper class for serialization."""
    def __init__(self, a):
        self.a = a

    def __reduce__(self):
        return A, (self.a.x,)

ray.get(ray.put(SerializationHelperForA(A(1))))  # success!
# the serializer only works for a specific object, not all A
# instances, so we still expect failure here.
ray.get(ray.put(A(1)))  # still fail!